In [72]:
#!pip install lightgbm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [73]:
train = pd.read_csv('train_amex/train.csv')
test = pd.read_csv('test_LNMuIYp/test.csv')

In [74]:
historical = pd.read_csv('train_amex/historical_user_logs.csv')

In [75]:
submission = pd.read_csv('sample_submission_2s8l9nF.csv')
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128858 entries, 0 to 128857
Data columns (total 2 columns):
session_id    128858 non-null int64
is_click      128858 non-null int64
dtypes: int64(2)
memory usage: 2.0 MB


In [76]:
train.drop(columns=['product_category_2','session_id','DateTime'], axis=1, inplace=True)
test.drop(columns=['product_category_2','session_id','DateTime'], axis=1, inplace=True)
#historical.drop(columns=['DateTime'],axis=1,inplace=True)

In [77]:
#train_hist=pd.merge(train,historical, how='left')

In [78]:
#Since there are more 0s in is_click as compared to 1s. There are only 31331 1s and 431960 0s. 
#Need to take the equal amount of values.

#train_1 = train_raw[train_raw['is_click']==1]
#train_0 = train_raw[train_raw['is_click']==0].head(300000)

#train = train_1.append(train_0)

In [79]:
#train_hist.info()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463291 entries, 0 to 463290
Data columns (total 12 columns):
user_id                   463291 non-null int64
product                   463291 non-null object
campaign_id               463291 non-null int64
webpage_id                463291 non-null int64
product_category_1        463291 non-null int64
user_group_id             445048 non-null float64
gender                    445048 non-null object
age_level                 445048 non-null float64
user_depth                445048 non-null float64
city_development_index    338162 non-null float64
var_1                     463291 non-null int64
is_click                  463291 non-null int64
dtypes: float64(4), int64(6), object(2)
memory usage: 42.4+ MB


In [80]:
#train_hist.head()
train.head()

,user_id,product,campaign_id,webpage_id,product_category_1,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click
0,858557,C,359520,13787,4,10.0,Female,4.0,3.0,3.0,0,0
1,243253,C,105960,11085,5,8.0,Female,2.0,2.0,NaN,0,0
2,243253,C,359520,13787,4,8.0,Female,2.0,2.0,NaN,0,0
3,1097446,I,359520,13787,3,3.0,Male,3.0,3.0,2.0,1,0
4,663656,C,405490,60305,3,2.0,Male,2.0,3.0,2.0,1,0


In [81]:
#test_hist=pd.merge(test,historical, how='left', on = ['product', 'user_id'] )

In [82]:
#historical = historical.drop_duplicates()
#train_hist = train_hist.drop_duplicates()
#test_hist = test_hist.drop_duplicates()

In [83]:
#test_hist.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128858 entries, 0 to 128857
Data columns (total 11 columns):
user_id                   128858 non-null int64
product                   128858 non-null object
campaign_id               128858 non-null int64
webpage_id                128858 non-null int64
product_category_1        128858 non-null int64
user_group_id             123174 non-null float64
gender                    123174 non-null object
age_level                 123174 non-null float64
user_depth                123174 non-null float64
city_development_index    94249 non-null float64
var_1                     128858 non-null int64
dtypes: float64(4), int64(5), object(2)
memory usage: 10.8+ MB


In [84]:
#test_hist.head()
test.head()

,user_id,product,campaign_id,webpage_id,product_category_1,user_group_id,gender,age_level,user_depth,city_development_index,var_1
0,732573,J,404347,53587,1,5.0,Male,5.0,3.0,NaN,0
1,172910,I,118601,28529,3,NaN,NaN,NaN,NaN,NaN,1
2,172910,I,118601,28529,4,NaN,NaN,NaN,NaN,NaN,1
3,557318,G,118601,28529,5,1.0,Male,1.0,3.0,1.0,0
4,923896,H,118601,28529,5,9.0,Female,3.0,1.0,NaN,1


In [85]:
historical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24287534 entries, 0 to 24287533
Data columns (total 4 columns):
DateTime    object
user_id     int64
product     object
action      object
dtypes: int64(1), object(3)
memory usage: 741.2+ MB


In [86]:
historical.head()

,DateTime,user_id,product,action
0,2017-05-28 15:44,704,B,view
1,2017-05-29 07:08,499679,F,view
2,2017-05-29 07:10,499679,G,view
3,2017-05-29 07:10,499679,G,view
4,2017-05-29 07:10,499679,G,view


In [87]:
#train_hist.dtypes
train.dtypes

user_id                     int64
product                    object
campaign_id                 int64
webpage_id                  int64
product_category_1          int64
user_group_id             float64
gender                     object
age_level                 float64
user_depth                float64
city_development_index    float64
var_1                       int64
is_click                    int64
dtype: object

In [88]:
train.groupby(['product'])['product'].count()

product
A     15391
B     22479
C    163501
D     41064
E     21452
F      7007
G      9414
H    109574
I     63711
J      9698
Name: product, dtype: int64

In [89]:
train = train.replace({'gender' : { 'Male' : 0, 'Female' : 1}})
test = test.replace({'gender' : { 'Male' : 0, 'Female' : 1}})

In [90]:
train = train.replace({'action' : { 'view' : 0, 'interest' : 1}})
test = test.replace({'action' : { 'view' : 0, 'interest' : 1}})

In [91]:
train=train.replace({'product': {'A': 0, 'B':1, 'C':2, "D":3,"E":4,"F":5,"G":6,"H":7,"I":8,"J":9}})
test=test.replace({'product': {'A': 0, 'B':1, 'C':2, "D":3,"E":4,"F":5,"G":6,"H":7,"I":8,"J":9}})

In [92]:
#converting datetime column into unix timestamp
#train_hist['DateTime']=pd.to_datetime(train_hist['DateTime'], format='%Y-%m-%d', utc=True)
#test_hist['DateTime']=pd.to_datetime(test_hist['DateTime'], format='%Y-%m-%d', utc=True)

In [93]:
train.groupby(['product_category_1'])['product_category_1'].count()

product_category_1
1     81141
2     69183
3    113812
4    133290
5     65865
Name: product_category_1, dtype: int64

In [94]:
#train_hist.groupby(['product_category_2'])['product_category_2'].count()

In [95]:
train.groupby(['gender'])['gender'].count()

gender
0.0    393454
1.0     51594
Name: gender, dtype: int64

In [96]:
train.groupby(['age_level'])['age_level'].count()

age_level
0.0       153
1.0     43367
2.0    143501
3.0    160581
4.0     63859
5.0     30828
6.0      2759
Name: age_level, dtype: int64

In [97]:
train.groupby(['user_depth'])['user_depth'].count()

user_depth
1.0     11861
2.0     30389
3.0    402798
Name: user_depth, dtype: int64

In [98]:
train.groupby(['city_development_index'])['city_development_index'].count()

city_development_index
1.0     34644
2.0    147643
3.0     88709
4.0     67166
Name: city_development_index, dtype: int64

In [99]:
train.groupby(['user_group_id'])['user_group_id'].count()

user_group_id
0.0        153
1.0      41946
2.0     137278
3.0     140317
4.0      50080
5.0      21989
6.0       1724
7.0       1421
8.0       6223
9.0      20264
10.0     13779
11.0      8839
12.0      1035
Name: user_group_id, dtype: int64

In [100]:
train.groupby(['var_1'])['var_1'].count()

var_1
0    267704
1    195587
Name: var_1, dtype: int64

In [101]:
train.groupby(['is_click'])['is_click'].count()

is_click
0    431960
1     31331
Name: is_click, dtype: int64

In [102]:
#train_hist= pd.merge(train,historical,how='inner',on=['user_id','product'])

In [103]:
#Imputing missing values

def missing_data_count(df):
  print(df.isna().sum())

missing_data_count(train)

user_id                        0
product                        0
campaign_id                    0
webpage_id                     0
product_category_1             0
user_group_id              18243
gender                     18243
age_level                  18243
user_depth                 18243
city_development_index    125129
var_1                          0
is_click                       0
dtype: int64


In [104]:
missing_data_count(test)

user_id                       0
product                       0
campaign_id                   0
webpage_id                    0
product_category_1            0
user_group_id              5684
gender                     5684
age_level                  5684
user_depth                 5684
city_development_index    34609
var_1                         0
dtype: int64


In [105]:
#useless columns - city_development_index
test["city_development_index"] = test["city_development_index"].fillna(test['city_development_index'].mode()[0])
test["user_depth"] = test["user_depth"].fillna(test['user_depth'].mode()[0])
test["age_level"] = test["age_level"].fillna(test['age_level'].mode()[0])
test["gender"] = test["gender"].fillna(test['gender'].mode()[0])
#user_group_id is highly correlated with the gender column
test["user_group_id"] = test["user_group_id"].fillna(test['user_group_id'].mode()[0])


train["city_development_index"] = train["city_development_index"].fillna(train['city_development_index'].mode()[0])
train["user_depth"] = train["user_depth"].fillna(train['user_depth'].mode()[0])
train["age_level"] = train["age_level"].fillna(train['age_level'].mode()[0])
train["gender"] = train["gender"].fillna(train['gender'].mode()[0])
train["user_group_id"] = train["user_group_id"].fillna(train['user_group_id'].mode()[0])

In [106]:
target = train['is_click']
target.value_counts()

train=train.drop(['is_click'],axis=1)
#test=test.drop(['product_category_2','session_id'],axis=1)

In [107]:
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

In [108]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

In [109]:
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.05)
clf.fit(X_train, y_train)
pred = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, pred)

0:	learn: 0.5859231	total: 351ms	remaining: 2m 55s
1:	learn: 0.5901724	total: 767ms	remaining: 3m 11s
2:	learn: 0.5925182	total: 1.27s	remaining: 3m 30s
3:	learn: 0.5962943	total: 1.74s	remaining: 3m 35s
4:	learn: 0.5980205	total: 2.14s	remaining: 3m 32s
5:	learn: 0.5985883	total: 2.52s	remaining: 3m 27s
6:	learn: 0.5987548	total: 2.91s	remaining: 3m 25s
7:	learn: 0.6007558	total: 3.3s	remaining: 3m 23s
8:	learn: 0.6013787	total: 3.68s	remaining: 3m 20s
9:	learn: 0.6026260	total: 4.07s	remaining: 3m 19s
10:	learn: 0.6031191	total: 4.51s	remaining: 3m 20s
11:	learn: 0.6040427	total: 4.9s	remaining: 3m 19s
12:	learn: 0.6047972	total: 5.29s	remaining: 3m 18s
13:	learn: 0.6057490	total: 5.66s	remaining: 3m 16s
14:	learn: 0.6073419	total: 6.12s	remaining: 3m 17s
15:	learn: 0.6078727	total: 6.52s	remaining: 3m 17s
16:	learn: 0.6086008	total: 6.91s	remaining: 3m 16s
17:	learn: 0.6086457	total: 7.3s	remaining: 3m 15s
18:	learn: 0.6097776	total: 7.68s	remaining: 3m 14s
19:	learn: 0.6108825	tota

158:	learn: 0.6591023	total: 1m 11s	remaining: 2m 32s
159:	learn: 0.6592526	total: 1m 11s	remaining: 2m 32s
160:	learn: 0.6593968	total: 1m 12s	remaining: 2m 31s
161:	learn: 0.6595663	total: 1m 12s	remaining: 2m 31s
162:	learn: 0.6596261	total: 1m 12s	remaining: 2m 30s
163:	learn: 0.6597660	total: 1m 13s	remaining: 2m 30s
164:	learn: 0.6600762	total: 1m 13s	remaining: 2m 29s
165:	learn: 0.6602458	total: 1m 13s	remaining: 2m 28s
166:	learn: 0.6604627	total: 1m 14s	remaining: 2m 28s
167:	learn: 0.6606161	total: 1m 14s	remaining: 2m 28s
168:	learn: 0.6608605	total: 1m 15s	remaining: 2m 27s
169:	learn: 0.6612106	total: 1m 16s	remaining: 2m 27s
170:	learn: 0.6614463	total: 1m 16s	remaining: 2m 27s
171:	learn: 0.6617407	total: 1m 17s	remaining: 2m 26s
172:	learn: 0.6619517	total: 1m 17s	remaining: 2m 26s
173:	learn: 0.6621893	total: 1m 17s	remaining: 2m 26s
174:	learn: 0.6624972	total: 1m 18s	remaining: 2m 25s
175:	learn: 0.6627189	total: 1m 18s	remaining: 2m 25s
176:	learn: 0.6629191	total:

311:	learn: 0.6879867	total: 2m 16s	remaining: 1m 22s
312:	learn: 0.6881345	total: 2m 16s	remaining: 1m 21s
313:	learn: 0.6882503	total: 2m 17s	remaining: 1m 21s
314:	learn: 0.6883998	total: 2m 17s	remaining: 1m 20s
315:	learn: 0.6884756	total: 2m 17s	remaining: 1m 20s
316:	learn: 0.6885675	total: 2m 18s	remaining: 1m 19s
317:	learn: 0.6887650	total: 2m 18s	remaining: 1m 19s
318:	learn: 0.6888797	total: 2m 19s	remaining: 1m 18s
319:	learn: 0.6890089	total: 2m 19s	remaining: 1m 18s
320:	learn: 0.6891231	total: 2m 20s	remaining: 1m 18s
321:	learn: 0.6891829	total: 2m 20s	remaining: 1m 17s
322:	learn: 0.6892519	total: 2m 21s	remaining: 1m 17s
323:	learn: 0.6893955	total: 2m 21s	remaining: 1m 16s
324:	learn: 0.6895405	total: 2m 22s	remaining: 1m 16s
325:	learn: 0.6898006	total: 2m 22s	remaining: 1m 16s
326:	learn: 0.6899381	total: 2m 23s	remaining: 1m 15s
327:	learn: 0.6900661	total: 2m 23s	remaining: 1m 15s
328:	learn: 0.6902039	total: 2m 23s	remaining: 1m 14s
329:	learn: 0.6902963	total:

467:	learn: 0.7083955	total: 3m 21s	remaining: 13.8s
468:	learn: 0.7084830	total: 3m 22s	remaining: 13.4s
469:	learn: 0.7085532	total: 3m 22s	remaining: 12.9s
470:	learn: 0.7085923	total: 3m 23s	remaining: 12.5s
471:	learn: 0.7087200	total: 3m 23s	remaining: 12.1s
472:	learn: 0.7087964	total: 3m 24s	remaining: 11.7s
473:	learn: 0.7089319	total: 3m 24s	remaining: 11.2s
474:	learn: 0.7090039	total: 3m 25s	remaining: 10.8s
475:	learn: 0.7090544	total: 3m 25s	remaining: 10.4s
476:	learn: 0.7092338	total: 3m 26s	remaining: 9.94s
477:	learn: 0.7092867	total: 3m 26s	remaining: 9.51s
478:	learn: 0.7093779	total: 3m 26s	remaining: 9.07s
479:	learn: 0.7094380	total: 3m 27s	remaining: 8.64s
480:	learn: 0.7096086	total: 3m 27s	remaining: 8.21s
481:	learn: 0.7098894	total: 3m 28s	remaining: 7.77s
482:	learn: 0.7099581	total: 3m 28s	remaining: 7.34s
483:	learn: 0.7100944	total: 3m 28s	remaining: 6.91s
484:	learn: 0.7101685	total: 3m 29s	remaining: 6.47s
485:	learn: 0.7102766	total: 3m 29s	remaining:

0.5756246020621587

In [110]:
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.05)
clf.fit(train, target)
pred = clf.predict_proba(test)[:,1]


0:	learn: 0.5855243	total: 497ms	remaining: 4m 8s
1:	learn: 0.5901278	total: 1.08s	remaining: 4m 28s
2:	learn: 0.5926690	total: 1.65s	remaining: 4m 33s
3:	learn: 0.5931817	total: 2.23s	remaining: 4m 36s
4:	learn: 0.5954090	total: 2.79s	remaining: 4m 36s
5:	learn: 0.5953911	total: 3.35s	remaining: 4m 35s
6:	learn: 0.5959582	total: 3.91s	remaining: 4m 35s
7:	learn: 0.5963593	total: 4.49s	remaining: 4m 36s
8:	learn: 0.5966860	total: 5.08s	remaining: 4m 37s
9:	learn: 0.5977528	total: 5.64s	remaining: 4m 36s
10:	learn: 0.5988720	total: 6.2s	remaining: 4m 35s
11:	learn: 0.5991057	total: 6.78s	remaining: 4m 35s
12:	learn: 0.6004981	total: 7.36s	remaining: 4m 35s
13:	learn: 0.6010823	total: 7.93s	remaining: 4m 35s
14:	learn: 0.6010428	total: 8.5s	remaining: 4m 34s
15:	learn: 0.6008909	total: 9.07s	remaining: 4m 34s
16:	learn: 0.6006874	total: 9.65s	remaining: 4m 34s
17:	learn: 0.6010374	total: 10.2s	remaining: 4m 33s
18:	learn: 0.6021878	total: 10.8s	remaining: 4m 34s
19:	learn: 0.6027427	tota

157:	learn: 0.6434830	total: 1m 41s	remaining: 3m 38s
158:	learn: 0.6436812	total: 1m 41s	remaining: 3m 38s
159:	learn: 0.6438323	total: 1m 42s	remaining: 3m 37s
160:	learn: 0.6441589	total: 1m 42s	remaining: 3m 36s
161:	learn: 0.6442570	total: 1m 43s	remaining: 3m 36s
162:	learn: 0.6443693	total: 1m 44s	remaining: 3m 35s
163:	learn: 0.6444997	total: 1m 44s	remaining: 3m 34s
164:	learn: 0.6447258	total: 1m 45s	remaining: 3m 34s
165:	learn: 0.6448544	total: 1m 46s	remaining: 3m 33s
166:	learn: 0.6449702	total: 1m 46s	remaining: 3m 33s
167:	learn: 0.6452507	total: 1m 47s	remaining: 3m 32s
168:	learn: 0.6454786	total: 1m 47s	remaining: 3m 31s
169:	learn: 0.6456195	total: 1m 48s	remaining: 3m 30s
170:	learn: 0.6456747	total: 1m 49s	remaining: 3m 30s
171:	learn: 0.6457689	total: 1m 49s	remaining: 3m 29s
172:	learn: 0.6459349	total: 1m 50s	remaining: 3m 29s
173:	learn: 0.6460617	total: 1m 51s	remaining: 3m 28s
174:	learn: 0.6461916	total: 1m 52s	remaining: 3m 28s
175:	learn: 0.6463644	total:

310:	learn: 0.6666026	total: 3m 27s	remaining: 2m 6s
311:	learn: 0.6666830	total: 3m 28s	remaining: 2m 5s
312:	learn: 0.6668108	total: 3m 29s	remaining: 2m 5s
313:	learn: 0.6669487	total: 3m 30s	remaining: 2m 4s
314:	learn: 0.6670918	total: 3m 30s	remaining: 2m 3s
315:	learn: 0.6671935	total: 3m 31s	remaining: 2m 3s
316:	learn: 0.6672600	total: 3m 32s	remaining: 2m 2s
317:	learn: 0.6674203	total: 3m 32s	remaining: 2m 1s
318:	learn: 0.6675051	total: 3m 33s	remaining: 2m 1s
319:	learn: 0.6676410	total: 3m 33s	remaining: 2m
320:	learn: 0.6677892	total: 3m 34s	remaining: 1m 59s
321:	learn: 0.6678905	total: 3m 35s	remaining: 1m 59s
322:	learn: 0.6680495	total: 3m 35s	remaining: 1m 58s
323:	learn: 0.6682487	total: 3m 36s	remaining: 1m 57s
324:	learn: 0.6683712	total: 3m 37s	remaining: 1m 56s
325:	learn: 0.6684501	total: 3m 37s	remaining: 1m 56s
326:	learn: 0.6686177	total: 3m 38s	remaining: 1m 55s
327:	learn: 0.6688103	total: 3m 38s	remaining: 1m 54s
328:	learn: 0.6688668	total: 3m 39s	remai

465:	learn: 0.6846316	total: 5m 15s	remaining: 23.1s
466:	learn: 0.6847112	total: 5m 16s	remaining: 22.4s
467:	learn: 0.6849126	total: 5m 17s	remaining: 21.7s
468:	learn: 0.6849388	total: 5m 17s	remaining: 21s
469:	learn: 0.6849965	total: 5m 18s	remaining: 20.3s
470:	learn: 0.6851081	total: 5m 19s	remaining: 19.6s
471:	learn: 0.6852010	total: 5m 19s	remaining: 19s
472:	learn: 0.6853235	total: 5m 20s	remaining: 18.3s
473:	learn: 0.6853796	total: 5m 21s	remaining: 17.6s
474:	learn: 0.6854858	total: 5m 22s	remaining: 17s
475:	learn: 0.6855230	total: 5m 22s	remaining: 16.3s
476:	learn: 0.6856040	total: 5m 23s	remaining: 15.6s
477:	learn: 0.6856578	total: 5m 23s	remaining: 14.9s
478:	learn: 0.6857102	total: 5m 24s	remaining: 14.2s
479:	learn: 0.6857429	total: 5m 24s	remaining: 13.5s
480:	learn: 0.6858032	total: 5m 25s	remaining: 12.9s
481:	learn: 0.6859373	total: 5m 26s	remaining: 12.2s
482:	learn: 0.6859588	total: 5m 26s	remaining: 11.5s
483:	learn: 0.6860500	total: 5m 27s	remaining: 10.8s

In [111]:
print(pred.shape)
submission["is_click"].shape

(128858,)


(128858,)

In [112]:
submission["is_click"]=pred
submission.to_csv("amexpert_hack_solution_catboost.csv", index=False)